# Import

In [ ]:
import os
import torch
# os.environ['TORCH'] = torch.__version__
# print(torch.__version__)


# !pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
# !pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

In [ ]:
import numpy as np
import warnings
import pandas as pd

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from google.colab import drive
from collections import Counter
from collections.abc import Iterable
import argparse
import os
import sys
import time
import pickle
import seaborn as sns
from tqdm import tqdm
import matplotlib.pyplot as plt
# import torch_geometric
from sklearn.metrics import r2_score

from torch.nn import Linear

# from torch_geometric.nn import GCNConv, TopKPooling, global_mean_pool, BatchNorm
# from torch_geometric.nn import global_mean_pool as gap, global_max_pool as gmp
# from torch_geometric.data import DataLoader as PYG_DataLoader
# from torch_geometric.data import Data
from itertools import cycle
import numpy as np
import warnings
import pandas as pd



drive.mount('/content/drive')

Mounted at /content/drive


# Load Data

In [ ]:
edge_path = "/content/drive/MyDrive/miRNA/graph/edge.pkl"
edge_attr_path = "/content/drive/MyDrive/miRNA/graph/edge_attr.pkl"
node_path = "/content/drive/MyDrive/miRNA/graph/node.pkl"
mesh_emb_path = '/content/drive/MyDrive/miRNA/embedding/mesh_embs.npy'
mesh_vocab_path = '/content/drive/MyDrive/miRNA/embedding/mesh_vocab.npy'

In [ ]:
all_x_disease_path = '/content/drive/MyDrive/miRNA/last_data/cir_X_Disease_v3.npy'
all_x_rna_path = '/content/drive/MyDrive/miRNA/last_data/cir_X_RNA_v3.npy'
all_pre_x_rna_path = '/content/drive/MyDrive/miRNA/last_data/cir_pre_RNA_x_v3.npy'
all_y_soft_path = '/content/drive/MyDrive/miRNA/last_data/cir_soft_y_v3.npy'
all_y_hard_path = '/content/drive/MyDrive/miRNA/last_data/cir_hard_y_v3.npy'
cir_hair_x_path = '/content/drive/MyDrive/miRNA/last_data/cir_pre_hair_X_v3.npy'
snd_x_path = "/content/drive/MyDrive/miRNA/last_data/cir_snd_X_v3.npy"

In [ ]:
all_orig_x_disease = np.load(all_x_disease_path, allow_pickle=True)
all_orig_x_rna = np.load(all_x_rna_path, allow_pickle=True)
all_orig_pre_x_rna = np.load(all_pre_x_rna_path, allow_pickle=True)

all_hair_x = np.load(cir_hair_x_path, allow_pickle=True)
snd_x = np.load(snd_x_path, allow_pickle=True)

all_orig_Y_soft = np.load(all_y_soft_path, allow_pickle=True)
all_orig_Y_hard = np.load(all_y_hard_path, allow_pickle=True)

In [ ]:
with open(edge_path, 'rb') as f:
    edge_d = pickle.load(f)

with open(edge_attr_path, 'rb') as f:
    edge_attr_d = pickle.load(f)

with open(node_path, 'rb') as f:
    node_d = pickle.load(f)

In [ ]:
lens = 0
item_num = 0
for n in node_d_1:
  item_num += 1
  lens += len(node_d_1[n])
print(lens/item_num)
print(item_num)

11.462222222222222
450


In [ ]:
# for regression
x_disease=[]
x_rna=[]
hair_x=[]
y_soft=[]
# y_hard=[]
max_len = 0
for i in range(len(all_orig_x_disease)):
  value = edge_d.get(all_orig_x_disease[i])
  if value != None:
    x_disease.append(all_orig_x_disease[i])
    # x_rna.append(all_orig_pre_x_rna[i]+all_orig_x_rna[i])
    x_rna.append(all_orig_x_rna[i])
    if max_len < len(all_orig_x_rna[i]):max_len = len(all_orig_x_rna[i])
    hair_x.append([*all_hair_x[i], *snd_x[i]])
    y_soft.append(all_orig_Y_soft[i])
    # # y_hard.append(all_orig_Y_hard[i])
    # 'down', 'ns', 'up'
    # print(i)

x_disease=np.array(x_disease)
x_rna=np.array(x_rna)
hair_x=np.array(hair_x)
y_soft=np.array(y_soft)
# # y_hard=np.array(y_hard)

In [ ]:
max_len

27

In [ ]:
print(len(all_orig_x_disease))
print(len(all_orig_x_rna))
print(len(all_orig_Y_soft))
print(len(all_orig_Y_hard))
print(len(all_orig_pre_x_rna))
print(len(all_hair_x))
print(len(snd_x))

7835
7835
7835
7835
7835
7835
7835


In [ ]:
print(len(x_disease))
print(len(x_rna))
print(len(y_soft))
# print(len(y_hard))
print(len(hair_x))

6587
6587
6587
6587


In [ ]:
keys = np.load(r'/content/drive/MyDrive/miRNA/embedding/vocab_npa.npy', allow_pickle=True)
values = np.load(r'/content/drive/MyDrive/miRNA/embedding/embs_npa.npy', allow_pickle=True)
word2vec = dict(zip(keys, values))

In [ ]:
a_file = open("/content/drive/MyDrive/miRNA/Circulation_data_seq/token.pkl", "rb")
disease_token_list = pickle.load(a_file)
print(disease_token_list)

{'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3, 'leukemia': 4, 'lymphocytic': 5, 'chronic': 6, 'b-cell': 7, 'colon': 8, 'neoplasms': 9, 'breast': 10, 'lung': 11, 'colorectal': 12, 'carcinoma': 13, 'promyelocytic': 14, 'acute': 15, '[unspecific]': 16, 'pituitary': 17, 'hepatocellular': 18, 'cardiomyopathy': 19, 'hypertrophic': 20, 'thyroid': 21, 'asthma': 22, 'lymphoma': 23, 'ovarian': 24, 'polycythemia': 25, 'vera': 26, 'myeloid': 27, 'lymphoblastic': 28, 'hodgkin': 29, 'nasopharyngeal': 30, 'retinal': 31, 'degeneration': 32, 'renal': 33, 'cell': 34, 'myelodysplastic': 35, 'syndromes': 36, 'neurodegenerative': 37, 'diseases': 38, 'lupus': 39, 'vulgaris': 40, 'chronic-phase': 41, 'leukemia-lymphoma': 42, 'precursor': 43, 'schizophrenia': 44, 'marek': 45, 'disease': 46, 'autistic': 47, 'disorder': 48, 'atherosclerosis': 49, 'sepsis': 50, 'choriocarcinoma': 51, 'glomerulonephritis': 52, 'child': 53, 'development': 54, 'disorders': 55, 'pervasive': 56, 'melanoma': 57, 'diabetes': 58, 'me

In [ ]:
disease_list = []
for i in range(len(x_disease)):
  disease = x_disease[i]
  disease = disease.replace(',', '')
  if disease not in disease_list: disease_list.append(disease)
disease_list

['Leukemia Lymphocytic Chronic B-Cell',
 'Colon Neoplasms',
 'Breast Neoplasms',
 'Leukemia',
 'Lung Neoplasms',
 'Colorectal Carcinoma',
 'Leukemia Promyelocytic Acute',
 'Neoplasms [unspecific]',
 'Pituitary Neoplasms',
 'Carcinoma Hepatocellular',
 'Cardiomyopathy Hypertrophic',
 'Thyroid Neoplasms',
 'Asthma',
 'Lymphoma B-Cell',
 'Ovarian Neoplasms',
 'Polycythemia Vera',
 'Leukemia Myeloid Acute',
 'Leukemia Lymphoblastic',
 'Lymphoma Hodgkin',
 'Nasopharyngeal Neoplasms',
 'Retinal Degeneration',
 'Carcinoma Renal Cell',
 'Myelodysplastic Syndromes',
 'Neurodegenerative Diseases [unspecific]',
 'Lupus Vulgaris',
 'Leukemia Myeloid Chronic-Phase',
 'Schizophrenia',
 'Marek Disease',
 'Autistic Disorder',
 'Atherosclerosis',
 'Sepsis',
 'Choriocarcinoma',
 'Glomerulonephritis',
 'Child Development Disorders Pervasive',
 'Melanoma',
 'Diabetes Mellitus Type 2',
 'Lymphoma Large-Cell Anaplastic',
 'Pancreatic Neoplasms',
 'Heart Failure',
 'Hemangioma',
 'Diabetes Mellitus',
 'Hyper

In [ ]:
lens = 0
word_list = []
for disease in disease_list:
  words = disease.split(' ')
  for word in words:
    word = word.lower()
    if word == "Toxic": print(words)
    if lens < len(words): lens = len(words)
    if word not in word_list: word_list.append(word)
lens

6

In [ ]:
convert_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
count = 3
for word in word_list:
  count += 1
  convert_dict[word] = count
convert_dict

{'1': 103,
 '2': 59,
 '[CLS]': 1,
 '[MASK]': 3,
 '[PAD]': 0,
 '[SEP]': 2,
 '[unspecific]': 16,
 'abdominal': 263,
 'acquired': 345,
 'acute': 15,
 'adenocarcinoma': 187,
 'adrenal': 189,
 'adrenocortical': 73,
 'adult': 71,
 'age-related': 250,
 'alcoholic': 310,
 'allergic': 130,
 'alzheimer': 75,
 'amyloidosis': 334,
 'amyotrophic': 197,
 'anaplastic': 61,
 'and': 156,
 'anemia': 151,
 'aneurysm': 262,
 'aneurysmal': 296,
 'angina': 217,
 'angiosarcoma': 235,
 'ankylosing': 315,
 'anxiety': 132,
 'aortic': 128,
 'aplastic': 249,
 'arrhythmia': 113,
 'arteriosclerosis': 256,
 'artery': 115,
 'arthritis': 110,
 'asthma': 22,
 'ataxia': 277,
 'atherosclerosis': 47,
 'atopic': 318,
 'atrial': 254,
 'atrophy': 135,
 'autism': 107,
 'autistic': 45,
 'autoimmune': 245,
 'azoospermia': 280,
 'b': 127,
 'b-cell': 7,
 'behcet': 153,
 'bicuspid': 275,
 'biliary': 94,
 'bladder': 79,
 'bone': 170,
 'bowel': 204,
 'brain': 146,
 'breast': 10,
 'bronchopulmonary': 232,
 'burkitt': 258,
 'c': 199,


In [ ]:
all_x_disease_path = '/content/drive/MyDrive/miRNA/last_data/all_X_Disease_v3.npy'
all_x_rna_path = '/content/drive/MyDrive/miRNA/last_data/all_X_RNA_v3.npy'
all_pre_x_rna_path = '/content/drive/MyDrive/miRNA/last_data/all_pre_RNA_x_v3.npy'
all_y_soft_path = '/content/drive/MyDrive/miRNA/last_data/all_soft_y_v3.npy'
all_y_hard_path = '/content/drive/MyDrive/miRNA/last_data/all_hard_y_v3.npy'
all_hair_x_path = '/content/drive/MyDrive/miRNA/last_data/all_pre_hair_X_v3.npy'
all_snd_x_path = "/content/drive/MyDrive/miRNA/last_data/all_snd_X_v3.npy"
all_orig_x_disease = np.load(all_x_disease_path, allow_pickle=True)
all_orig_x_rna = np.load(all_x_rna_path, allow_pickle=True)
all_orig_pre_x_rna = np.load(all_pre_x_rna_path, allow_pickle=True)

all_all_hair_x = np.load(all_hair_x_path, allow_pickle=True)
all_snd_x = np.load(all_snd_x_path, allow_pickle=True)

all_orig_Y_soft = np.load(all_y_soft_path, allow_pickle=True)
all_orig_Y_hard = np.load(all_y_hard_path, allow_pickle=True)
print(len(all_orig_x_disease))
print(len(all_orig_x_rna))
print(len(all_orig_Y_soft))
print(len(all_orig_Y_hard))
print(len(all_orig_pre_x_rna))
print(len(all_all_hair_x))
print(len(all_snd_x))

20852
20852
20852
20852
20852
20852
20852


In [ ]:
# all for regression

all_x_disease=[]
all_x_rna=[]
all_hair_x=[]
all_y_soft=[]
# y_hard=[]
max_len = 0
for i in range(len(all_orig_x_disease)):
  value = edge_d.get(all_orig_x_disease[i])
  if value != None:
    all_x_disease.append(all_orig_x_disease[i])
    # x_rna.append(all_orig_pre_x_rna[i]+all_orig_x_rna[i])
    all_x_rna.append(all_orig_x_rna[i])
    if max_len < len(all_orig_x_rna[i]):max_len = len(all_orig_x_rna[i])
    all_hair_x.append([*all_all_hair_x[i], *all_snd_x[i]])
    all_y_soft.append(all_orig_Y_soft[i])
    # # y_hard.append(all_orig_Y_hard[i])
    # 'down', 'ns', 'up'
    # print(i)

all_x_disease=np.array(all_x_disease)
all_x_rna=np.array(all_x_rna)
all_hair_x=np.array(all_hair_x)
all_y_soft=np.array(all_y_soft)
# # y_hard=np.array(y_hard)

In [ ]:
print(len(all_x_disease))
print(len(all_x_rna))
print(len(all_y_soft))
print(len(all_hair_x))

16912
16912
16912
16912


# Dataset

In [ ]:
class RegularDataset(Dataset):
  def __init__(self, orig_diseases, orig_mirna, orig_labels, hair_x, if_bert=False, disease_token_list=None):
    super(RegularDataset, self).__init__()
    self.orig_diseases = orig_diseases
    self.orig_mirna = orig_mirna
    self.orig_labels = orig_labels
    self.vocab_size = 5
    self.MAX_LEN = 27
    self.MAX_LEN_D = 6
    self.hair_x = hair_x
    self.if_bert = if_bert
    self.disease_token_list = disease_token_list
    self.letter2Number = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3, "A": 4, "C": 5, "G": 6, "U": 7}

  def convert_letter_to_number(self, lettter):
    letter_to_number = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3, "A": 4, "C": 5, "G": 6, "U": 7}

    return letter_to_number[lettter]


  def make_data_with_unified_length(self, token_list, max_len, covert_dict):
      max_len = max_len + 2  # add [CLS] and [SEP]

      token_list = [covert_dict['[CLS]']] + token_list + [covert_dict['[SEP]']]
      
      n_pad = max_len - len(token_list)
      token_list.extend([0] * n_pad)

      return token_list

  def mirna2list(self, data):
    out = []
    for j in range(len(data)):
      coverted_rna = self.convert_letter_to_number(data[j])
      out.append(coverted_rna)

    data = self.make_data_with_unified_length(out, self.MAX_LEN, self.letter2Number)
    
    return np.array(data)
  
  def disease2list(self, disease):
    disease = disease.replace(",", "")
    disease = disease.replace("-", " ")
    disease = disease.replace("[", "")
    disease = disease.replace("]", "")
    disease = disease.replace("Kideny", "Kidney")
    disease = disease.replace("Spondylarthritis", "Spondyloarthritis")
    disease_list = disease.split(" ")
    return disease_list


  def disease2emb(self, disease):

    disease_list = self.disease2list(disease)
    vector = np.zeros(300)
    for word in disease_list:
      value = word2vec.get(word)
      if value is not None:
        vector += word2vec[word]
      else:
        vector = np.zeros(300)
    out = vector/len(disease_list)
    return out

  def disease2token(self, disease):
  
    disease = disease.replace(", ", " ")
    disease_list = disease.split(" ")
    
    # print(disease_list)
    tokenized=[self.disease_token_list[i] for i in disease_list]
    max_len = self.MAX_LEN_D + 2
    tokenized = [self.disease_token_list['[CLS]']] + tokenized + [self.disease_token_list['[SEP]']]
    n_pad = max_len - len(tokenized)
    tokenized.extend([0] * n_pad)
    return np.array(tokenized)

  def __getitem__(self, index):
      orig_disease = self.orig_diseases[index]
      orig_mirna = self.orig_mirna[index]
      label = self.orig_labels[index]+1
      mirna = self.mirna2list(orig_mirna)
      pre_hair_x = torch.from_numpy(np.array([*self.hair_x[index], *self.mirna2list(orig_mirna)])).float()
      if self.if_bert:
        disease = orig_disease.lower()
        disease = self.disease2token(disease)
      else:
        disease = self.disease2emb(orig_disease)
        disease = np.expand_dims(disease, axis=0)
      return disease, mirna, label, pre_hair_x, orig_disease, orig_mirna
      

  def __len__(self):
    return len(self.orig_diseases)


In [ ]:
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x, if_bert=True, disease_token_list=disease_token_list)
dataloader = DataLoader(dataset, batch_size=2)

for idx, (disease, mirna, label, pre_hair_x, orig_disease, orig_mirna) in enumerate(dataloader):
  print(idx)
  print('disease.shape ',disease.shape)
  print('disease ',disease)
  print('pre_hair_x.shape ',pre_hair_x.shape)
  # print(mirna.shape)
  print('label.shape ',label.shape)
  print('label ',label)
  print('orig_disease ',orig_disease)
  print('orig_mirna ',orig_mirna)

  break

0
disease.shape  torch.Size([2, 8])
disease  tensor([[1, 4, 5, 6, 7, 2, 0, 0],
        [1, 4, 5, 6, 7, 2, 0, 0]])
pre_hair_x.shape  torch.Size([2, 70])
label.shape  torch.Size([2])
label  tensor([0.6000, 0.6000], dtype=torch.float64)
orig_disease  ('Leukemia, Lymphocytic, Chronic, B-Cell', 'Leukemia, Lymphocytic, Chronic, B-Cell')
orig_mirna  ('UAGCAGCACAUAAUGGUUUGUG', 'CAGGCCAUAUUGUGCUGCCUCA')


In [ ]:
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x)
dataloader = DataLoader(dataset, batch_size=2)

for idx, (disease, mirna, label, pre_hair_x, orig_disease, orig_mirna) in enumerate(dataloader):
  print(idx)
  print('disease.shape ',disease.shape)
  print('pre_hair_x.shape ',pre_hair_x.shape)
  # print(mirna.shape)
  print('label.shape ',label.shape)
  print('label ',label)
  print('orig_disease ',orig_disease)
  print('orig_mirna ',orig_mirna)

  break

0
disease.shape  torch.Size([2, 1, 300])
pre_hair_x.shape  torch.Size([2, 70])
label.shape  torch.Size([2])
label  tensor([0.6000, 0.6000], dtype=torch.float64)
orig_disease  ('Leukemia, Lymphocytic, Chronic, B-Cell', 'Leukemia, Lymphocytic, Chronic, B-Cell')
orig_mirna  ('UAGCAGCACAUAAUGGUUUGUG', 'CAGGCCAUAUUGUGCUGCCUCA')


# Model

## Diseases

### BERT

In [ ]:
def get_train_config_disease():
    parse = argparse.ArgumentParser(description='iDNA_ABT train model')

    # preoject setting
    parse.add_argument('-learn-name', type=str, default='iDNA_ABT_disease_train', help='learn name')
    parse.add_argument('-save-best', type=bool, default=False, help='if save parameters of the current best model ')
    parse.add_argument('-threshold', type=float, default=0.80, help='save threshold')
    
    # model parameters
    parse.add_argument('-vocab_size', type=int, default=424, help='dictionary size of letter to token')
    parse.add_argument('-max-len', type=int, default=8, help='max length of input sequences')
    parse.add_argument('-num-layer', type=int, default=3, help='number of encoder blocks')  # 3
    parse.add_argument('-num-head', type=int, default=8, help='number of head in multi-head attention')  # 8
    parse.add_argument('-dim-embedding', type=int, default=64, help='residue embedding dimension')  # 64
    parse.add_argument('-dim-feedforward', type=int, default=64, help='hidden layer dimension in feedforward layer')
    parse.add_argument('-dim-k', type=int, default=32, help='embedding dimension of vector k or q')
    parse.add_argument('-dim-v', type=int, default=32, help='embedding dimension of vector v')
    parse.add_argument('-num-embedding', type=int, default=2, help='number of sense in multi-sense')
    parse.add_argument('-k-mer', type=int, default=3, help='number of k(-mer) in multi-sccaled')
    parse.add_argument('-embed-atten-size', type=int, default=8, help='size of soft attention')

    # training parameters
    parse.add_argument('-lr', type=float, default=0.0005, help='learning rate')
    parse.add_argument('-reg', type=float, default=0.0025, help='weight lambda of regularization')
    parse.add_argument('-batch-size', type=int, default=64, help='number of samples in a batch')
    parse.add_argument('-epoch', type=int, default=100, help='number of iteration')  # 30
    parse.add_argument('-k-fold', type=int, default=-1, help='k in cross validation,-1 represents train-test approach')
    parse.add_argument('-num-class', type=int, default=3, help='number of classes')
    parse.add_argument('-b', type=int, default=0.06, help='b')
    # parse.add_argument('-cuda', type=bool, default=True, help='if use cuda')
    parse.add_argument('-cuda', type=bool, default=False, help='if not use cuda')
    parse.add_argument('-device', type=int, default=0, help='device id')
    parse.add_argument('-interval-log', type=int, default=10,
                       help='how many batches have gone through to record the training performance')
    parse.add_argument('-interval-valid', type=int, default=5,
                       help='how many epoches have gone through to record the validation performance')  # 20
    parse.add_argument('-interval-test', type=int, default=1,
                       help='how many epoches have gone through to record the test performance')
    parse.add_argument('-alpha', type=float, default=0.1, help='information entropy')

    config = parse.parse_args(args=[])

    return config

In [ ]:
def set_freeze_by_names(model, layer_names, freeze=True):
    if not isinstance(layer_names, Iterable):
        layer_names = [layer_names]
    for name, child in model.named_children():
        if name not in layer_names:
            continue
        for param in child.parameters():
            param.requires_grad = not freeze


def freeze_by_names(model, layer_names):
    set_freeze_by_names(model, layer_names, True)


def unfreeze_by_names(model, layer_names):
    set_freeze_by_names(model, layer_names, False)


def set_freeze_by_idxs(model, idxs, freeze=True):
    if not isinstance(idxs, Iterable):
        idxs = [idxs]
    num_child = len(list(model.children()))
    idxs = tuple(map(lambda idx: num_child + idx if idx < 0 else idx, idxs))
    for idx, child in enumerate(model.children()):
        if idx not in idxs:
            continue
        for param in child.parameters():
            param.requires_grad = not freeze


def freeze_by_idxs(model, idxs):
    set_freeze_by_idxs(model, idxs, True)


def unfreeze_by_idxs(model, idxs):
    set_freeze_by_idxs(model, idxs, False)

In [ ]:
def get_attn_pad_mask(seq):
    batch_size, seq_len = seq.size()
    pad_attn_mask = seq.data.eq(0).unsqueeze(1)  # [batch_size, 1, seq_len]
    pad_attn_mask_expand = pad_attn_mask.expand(batch_size, seq_len, seq_len)  # [batch_size, seq_len, seq_len]
    return pad_attn_mask_expand


class Embedding(nn.Module):
    def __init__(self, config):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding (look-up table) [64, 43, 64]
        self.pos_embed = nn.Embedding(max_len, d_model)  # position embedding [64, 43, 64]
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        seq_len = x.size(1)  # x: [batch_size, seq_len]
        
        pos = torch.arange(seq_len, device=device, dtype=torch.long)  # [seq_len]

        pos = pos.unsqueeze(0).expand_as(x)  # [seq_len] -> [batch_size, seq_len]
        embedding = self.pos_embed(pos)
        embedding = embedding + self.tok_embed(x)
        embedding = self.norm(embedding)
        return embedding


class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)  # scores : [batch_size, n_head, seq_len, seq_len]
        scores.masked_fill_(attn_mask, -1e9)  # Fills elements of self tensor with value where mask is one.
        attn = nn.Softmax(dim=-1)(scores)  # [batch_size, n_head, seq_len, seq_len]
        context = torch.matmul(attn, V)  # [batch_size, n_head, seq_len, d_v]
        return context, attn


class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_head)
        self.W_K = nn.Linear(d_model, d_k * n_head)
        self.W_V = nn.Linear(d_model, d_v * n_head)

        self.linear = nn.Linear(n_head * d_v, d_model)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, Q, K, V, attn_mask):
        residual, batch_size = Q, Q.size(0)
        q_s = self.W_Q(Q).view(batch_size, -1, n_head, d_k).transpose(1, 2)  # q_s: [batch_size, n_head, seq_len, d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_head, d_k).transpose(1, 2)  # k_s: [batch_size, n_head, seq_len, d_k]
        v_s = self.W_V(V).view(batch_size, -1, n_head, d_v).transpose(1, 2)  # v_s: [batch_size, n_head, seq_len, d_v]
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_head, 1, 1)
        context, attention_map = ScaledDotProductAttention()(q_s, k_s, v_s, attn_mask)
        context = context.transpose(1, 2).contiguous().view(batch_size, -1,
                                                            n_head * d_v)  # context: [batch_size, seq_len, n_head * d_v]
        output = self.linear(context)
        output = self.norm(output + residual)
        return output, attention_map


class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        # (batch_size, seq_len, d_model) -> (batch_size, seq_len, d_ff) -> (batch_size, seq_len, d_model)
        return self.fc2(self.relu(self.fc1(x)))


class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()
        self.attention_map = None

    def forward(self, enc_inputs, enc_self_attn_mask):
        enc_outputs, attention_map = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs,
                                                        enc_self_attn_mask)  # enc_inputs to same Q,K,V
        self.attention_map = attention_map
        enc_outputs = self.pos_ffn(enc_outputs)  # enc_outputs: [batch_size, seq_len, d_model]
        return enc_outputs


class BERT(nn.Module):
    def __init__(self, config):
        super(BERT, self).__init__()

        global max_len, n_layers, n_head, d_model, d_ff, d_k, d_v, vocab_size, device
        max_len = config.max_len
        n_layers = config.num_layer
        n_head = config.num_head
        d_model = config.dim_embedding
        d_ff = config.dim_feedforward
        d_k = config.dim_k
        d_v = config.dim_v
        vocab_size = config.vocab_size
        device = torch.device("cuda")

        self.embedding = Embedding(config)
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc_task = nn.Sequential(
            nn.Linear(d_model, d_model // 2),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(d_model // 2, 2),
        )
        self.classifier = nn.Linear(2, 2)

    def forward(self, input_ids):
        output = self.embedding(input_ids)  # [bach_size, seq_len, d_model]
        enc_self_attn_mask = get_attn_pad_mask(input_ids)  # [batch_size, maxlen, maxlen]
        for layer in self.layers:
            output = layer(output, enc_self_attn_mask)
            # output: [batch_size, max_len, d_model]

        # classification
        # only use [CLS]
        representation = output[:, 0, :]
        # print(representation.shape)
        reduction_feature = self.fc_task(representation)
        reduction_feature = reduction_feature.view(reduction_feature.size(0), -1)
        logits_clsf = self.classifier(reduction_feature)
        # representation = reduction_feature
        return logits_clsf, representation

In [ ]:
# class Classificaion_module(nn.Module):
#     def __init__(self, config_disease, tanhshrink_ratio=3):
#       super(Classificaion_module, self).__init__()
#       self.tanhshrink_ratio = tanhshrink_ratio
#       self.bert_disease_module = BERT(config_disease)
#       self.FC1 = nn.Linear(94, 32)
#       self.RELU = nn.ReLU()
#       self.FC2 = nn.Linear(32, 1)
#       self.dropout = nn.Dropout(0.5)


#     def forward(self, x_d, x_mr):
#       # print('x_d type',type(x_d))
#       # print('x_mr type',type(x_mr))
#       _, d_di = self.bert_disease_module(x_d)
#       # print('x_d.shape',x_d.shape)
#       # print('x_mr.shape',x_mr.shape)
#       # print('d_di.shape',d_di.shape)
#       # print('d_mr.shape',d_mr.shape)
#       x = torch.cat((x_mr, d_di), dim=1)
#       # print(x.shape)
#       x = self.FC1(x)
#       x = self.RELU(x)
#       x = self.dropout(x)
#       x = self.FC2(x)

#       return x



In [ ]:
# from torch.nn.modules.batchnorm import BatchNorm1d
class Classificaion_module(nn.Module):
    def __init__(self, config_disease):
      super(Classificaion_module, self).__init__()
      
      self.bert_disease_module = BERT(config_disease)
      # self.conv1d_module = Conv1d_module()
      self.linear_1 = nn.Linear(70, 128)
      self.bn_1 = nn.BatchNorm1d(128)
      self.linear_2 = nn.Linear(128, 256)
      self.bn_2 = nn.BatchNorm1d(256)
      self.linear_3 = nn.Linear(256, 128)
      self.bn_3 = nn.BatchNorm1d(128)
      self.linear_4 = nn.Linear(128, 64)
      self.bn_4 = nn.BatchNorm1d(64)
      self.dropout = nn.Dropout(0.2)

      self.FC1 = nn.Linear(128, 64)
      self.RELU = nn.ReLU()
      self.fc_bn_1 = nn.BatchNorm1d(64)
      self.FC2 = nn.Linear(64, 32)
      self.fc_bn_2 = nn.BatchNorm1d(32)
      self.FC3 = nn.Linear(32, 1)
      


    def forward(self, x_d, x_mr):

      # d_d = self.lstm_module(x_d)
      _, d_d = self.bert_disease_module(x_d)
      d_mr = self.linear_1(x_mr)
      d_mr = self.bn_1(d_mr)
      d_mr = self.RELU(d_mr)
      # d_mr = self.dropout(d_mr)
      d_mr = self.linear_2(d_mr)
      d_mr = self.bn_2(d_mr)
      d_mr = self.RELU(d_mr)
      d_mr = self.dropout(d_mr)
      d_mr = self.linear_3(d_mr)
      d_mr = self.bn_3(d_mr)
      d_mr = self.RELU(d_mr)
      # d_mr = self.dropout(d_mr)
      d_mr = self.linear_4(d_mr)
      d_mr = self.bn_4(d_mr)
      d_mr = self.RELU(d_mr)
      # d_mr = self.conv1d_module(data.pre_hair_x)

      x = torch.cat((d_mr, d_d), dim=1)
      # print(x.shape)
      x = self.FC1(x)
      x = self.fc_bn_1(x)
      x = self.RELU(x)
      x = self.FC2(x)
      x = self.fc_bn_2(x)
      x = self.RELU(x)
      x = self.FC3(x)

      return x



In [ ]:
config_d = get_train_config_disease()

model_class = Classificaion_module(config_d).cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x, if_bert=True, disease_token_list=convert_dict)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-300, 300])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=32)
val_dataloader = DataLoader(val_dataset, batch_size=32)

for idx, (disease, mirna, label, pre_hair_x, orig_disease, orig_mirna) in enumerate(dataloader):
  print(pre_hair_x.shape)
  pre_hair_x = pre_hair_x.cuda()
  disease = disease.cuda()
  print(disease.shape)
  output = model_class(disease, pre_hair_x)
  print('output.shape', output.shape)

  break

torch.Size([32, 70])
torch.Size([32, 8])
output.shape torch.Size([32, 1])


### LSTM

In [ ]:
class LSTM_module(nn.Module):
  def __init__(self):
    super(LSTM_module, self).__init__()
    self.hidden_size = 64
    self.embedding_dim = 300
    self.num_layer = 2
    self.bidirectional = True
    self.bi_num = 2 if self.bidirectional else 1
    self.dropout = 0.5
    # input(seq_len, batch, input_size)
    # h0(num_layers * num_directions, batch, hidden_size)
    # c0(num_layers * num_directions, batch, hidden_size)
    self.lstm1 = nn.LSTM(self.embedding_dim, self.hidden_size, self.num_layer, bidirectional=True, dropout=self.dropout)
    # self.lstm2 = nn.LSTM(128, 64, self.num_layer, bidirectional=True, dropout=self.dropout)
    self.FC = nn.Linear(128, 64)

  def forward(self, x):
    h_0, c_0 = self.init_hidden_state(x.size(0))
    # x = np.transpose(x, (1,0,2))
    x = torch.swapaxes(x, 0, 1)
    x = x.to(torch.float32)
    # print('h_0', h_0.dtype)
    # print('c_0', c_0.dtype)
    # print('x', x.dtype)
    x, (h_n, c_n) = self.lstm1(x, (h_0, c_0))
    # print('x', x.shape)
    # x, (h_n, c_n) = self.lstm2(x, (h_n, c_n))
    # print('output', output)
    # print('h_n', h_n.shape)
    # print('c_n', c_n.shape)
    out = torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=-1)
    # print(out)
    out = self.FC(out)
    return out
  
  def init_hidden_state(self, batch_size):
    h_0 = torch.rand(self.num_layer * self.bi_num, batch_size, self.hidden_size).to('cuda:0')
    c_0 = torch.rand(self.num_layer * self.bi_num, batch_size, self.hidden_size).to('cuda:0')
    return h_0, c_0

In [ ]:
# from torch.nn.modules.batchnorm import BatchNorm1d
class Classificaion_module(nn.Module):
    def __init__(self):
      super(Classificaion_module, self).__init__()
      
      self.lstm_module = LSTM_module()
      # self.conv1d_module = Conv1d_module()
      self.linear_1 = nn.Linear(70, 128)
      self.bn_1 = nn.BatchNorm1d(128)
      self.linear_2 = nn.Linear(128, 256)
      self.bn_2 = nn.BatchNorm1d(256)
      self.linear_3 = nn.Linear(256, 128)
      self.bn_3 = nn.BatchNorm1d(128)
      self.linear_4 = nn.Linear(128, 64)
      self.bn_4 = nn.BatchNorm1d(64)
      self.dropout = nn.Dropout(0.2)

      self.FC1 = nn.Linear(128, 64)
      self.RELU = nn.ReLU()
      self.fc_bn_1 = nn.BatchNorm1d(64)
      self.FC2 = nn.Linear(64, 32)
      self.fc_bn_2 = nn.BatchNorm1d(32)
      self.FC3 = nn.Linear(32, 1)
      


    def forward(self, x_d, x_mr):

      d_d = self.lstm_module(x_d)
      d_mr = self.linear_1(x_mr)
      d_mr = self.bn_1(d_mr)
      d_mr = self.RELU(d_mr)
      # d_mr = self.dropout(d_mr)
      d_mr = self.linear_2(d_mr)
      d_mr = self.bn_2(d_mr)
      d_mr = self.RELU(d_mr)
      d_mr = self.dropout(d_mr)
      d_mr = self.linear_3(d_mr)
      d_mr = self.bn_3(d_mr)
      d_mr = self.RELU(d_mr)
      # d_mr = self.dropout(d_mr)
      d_mr = self.linear_4(d_mr)
      d_mr = self.bn_4(d_mr)
      d_mr = self.RELU(d_mr)
      # d_mr = self.conv1d_module(data.pre_hair_x)

      x = torch.cat((d_mr, d_d), dim=1)
      # print(x.shape)
      x = self.FC1(x)
      x = self.fc_bn_1(x)
      x = self.RELU(x)
      x = self.FC2(x)
      x = self.fc_bn_2(x)
      x = self.RELU(x)
      x = self.FC3(x)

      return x



In [ ]:

model_class = Classificaion_module().cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-300, 300])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=32)
val_dataloader = DataLoader(val_dataset, batch_size=32)

for idx, (disease, mirna, label, pre_hair_x, orig_disease, orig_mirna) in enumerate(dataloader):
  print(pre_hair_x.shape)
  pre_hair_x = pre_hair_x.cuda()
  disease = disease.to(torch.float32).cuda()
  print(disease.shape)
  output = model_class(disease, pre_hair_x)
  print('output.shape', output.shape)

  break

torch.Size([32, 70])
torch.Size([32, 1, 300])
output.shape torch.Size([32, 1])


### GRU

In [ ]:
class GRU_module(nn.Module):
  def __init__(self):
    super(GRU_module, self).__init__()
    self.hidden_size = 64
    self.embedding_dim = 300
    self.num_layer = 2
    self.bidirectional = True
    self.bi_num = 2 if self.bidirectional else 1
    self.dropout = 0.5
    # input(seq_len, batch, input_size)
    # h0(num_layers * num_directions, batch, hidden_size)
    # c0(num_layers * num_directions, batch, hidden_size)
    self.gru = nn.GRU(self.embedding_dim, self.hidden_size, self.num_layer, bidirectional=True, dropout=self.dropout)
    # self.lstm2 = nn.LSTM(128, 64, self.num_layer, bidirectional=True, dropout=self.dropout)
    self.FC = nn.Linear(128, 64)

  def forward(self, x):
    h_0, c_0 = self.init_hidden_state(x.size(0))
    # x = np.transpose(x, (1,0,2))
    x = torch.swapaxes(x, 0, 1)
    x = x.to(torch.float32)
    # print('h_0', h_0.dtype)
    # print('c_0', c_0.dtype)
    # print('x', x.dtype)
    x, h_n = self.gru(x, h_0)
    # print('x', x.shape)
    # x, (h_n, c_n) = self.lstm2(x, (h_n, c_n))
    # print('output', output)
    # print('h_n', h_n.shape)
    # print('c_n', c_n.shape)
    out = torch.cat((h_n[-2, :, :], h_n[-1, :, :]), dim=-1)
    # print(out)
    out = self.FC(out)
    return out
  
  def init_hidden_state(self, batch_size):
    h_0 = torch.rand(self.num_layer * self.bi_num, batch_size, self.hidden_size).to('cuda:0')
    c_0 = torch.rand(self.num_layer * self.bi_num, batch_size, self.hidden_size).to('cuda:0')
    return h_0, c_0

In [ ]:
# from torch.nn.modules.batchnorm import BatchNorm1d
class Classificaion_module(nn.Module):
    def __init__(self):
      super(Classificaion_module, self).__init__()
      
      self.gru_module = GRU_module()
      # self.conv1d_module = Conv1d_module()
      self.linear_1 = nn.Linear(70, 128)
      self.bn_1 = nn.BatchNorm1d(128)
      self.linear_2 = nn.Linear(128, 256)
      self.bn_2 = nn.BatchNorm1d(256)
      self.linear_3 = nn.Linear(256, 128)
      self.bn_3 = nn.BatchNorm1d(128)
      self.linear_4 = nn.Linear(128, 64)
      self.bn_4 = nn.BatchNorm1d(64)
      self.dropout = nn.Dropout(0.2)

      self.FC1 = nn.Linear(128, 64)
      self.RELU = nn.ReLU()
      self.fc_bn_1 = nn.BatchNorm1d(64)
      self.FC2 = nn.Linear(64, 32)
      self.fc_bn_2 = nn.BatchNorm1d(32)
      self.FC3 = nn.Linear(32, 1)
      


    def forward(self, x_d, x_mr):

      d_d = self.gru_module(x_d)
      d_mr = self.linear_1(x_mr)
      d_mr = self.bn_1(d_mr)
      d_mr = self.RELU(d_mr)
      d_mr = self.dropout(d_mr)
      d_mr = self.linear_2(d_mr)
      d_mr = self.bn_2(d_mr)
      d_mr = self.RELU(d_mr)
      d_mr = self.dropout(d_mr)
      d_mr = self.linear_3(d_mr)
      d_mr = self.bn_3(d_mr)
      d_mr = self.RELU(d_mr)
      # d_mr = self.dropout(d_mr)
      d_mr = self.linear_4(d_mr)
      d_mr = self.bn_4(d_mr)
      d_mr = self.RELU(d_mr)
      # d_mr = self.conv1d_module(data.pre_hair_x)

      x = torch.cat((d_mr, d_d), dim=1)
      # print(x.shape)
      x = self.FC1(x)
      x = self.fc_bn_1(x)
      x = self.RELU(x)
      x = self.FC2(x)
      x = self.fc_bn_2(x)
      x = self.RELU(x)
      x = self.FC3(x)

      return x



In [ ]:

model_class = Classificaion_module().cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-300, 300])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=32)
val_dataloader = DataLoader(val_dataset, batch_size=32)

for idx, (disease, mirna, label, pre_hair_x, orig_disease, orig_mirna) in enumerate(dataloader):
  print(pre_hair_x.shape)
  pre_hair_x = pre_hair_x.cuda()
  disease = disease.to(torch.float32).cuda()
  print(disease.shape)
  output = model_class(disease, pre_hair_x)
  print('output.shape', output.shape)

  break

torch.Size([32, 70])
torch.Size([32, 1, 300])
output.shape torch.Size([32, 1])


# Train

## BERT

In [ ]:
config_d = get_train_config_disease()

model_class = Classificaion_module(config_d).cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x, if_bert=True, disease_token_list=convert_dict)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-300, 300])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=32)
val_dataloader = DataLoader(val_dataset, batch_size=32)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
optimizer = torch.optim.AdamW(params=model_class.parameters(), lr=0.0008, weight_decay=0.0001)
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
criterion_L1 = nn.L1Loss()

In [ ]:
model_class.train()
model_class

Classificaion_module(
  (bert_disease_module): BERT(
    (embedding): Embedding(
      (tok_embed): Embedding(424, 64)
      (pos_embed): Embedding(8, 64)
      (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (layers): ModuleList(
      (0): EncoderLayer(
        (enc_self_attn): MultiHeadAttention(
          (W_Q): Linear(in_features=64, out_features=256, bias=True)
          (W_K): Linear(in_features=64, out_features=256, bias=True)
          (W_V): Linear(in_features=64, out_features=256, bias=True)
          (linear): Linear(in_features=256, out_features=64, bias=True)
          (norm): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
        )
        (pos_ffn): PoswiseFeedForwardNet(
          (fc1): Linear(in_features=64, out_features=64, bias=True)
          (fc2): Linear(in_features=64, out_features=64, bias=True)
          (relu): ReLU()
        )
      )
      (1): EncoderLayer(
        (enc_self_attn): MultiHeadAttention(
          (W_Q): Linear(i

In [ ]:
def test_acc_output(epoch):
  corrects = 0
  the_batch_size = 0

  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0

  # state = default_evaluator.run([[y_pred, y_true]])
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(test_dataloader):
    disease = disease.cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    digits = model_class(disease, pre_hair_x)
    # digits = F.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)
    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTesting Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1
  # corrects = (torch.max(logits, 1)[1] == label).sum()

In [ ]:
def val_acc_output(epoch):
  corrects = 0
  the_batch_size = 0
  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(val_dataloader):
    disease = disease.cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    digits = model_class(disease, pre_hair_x)

    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)

    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTesting Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1

In [ ]:
steps = 0
best_acc = 0
best_performance = 0
step_log_interval = []
train_loss_record = []
train_loss_l1_record = []

val_acc_record = []
val_loss_record = []
val_r2_record = []
val_loss_l1_record = []

test_acc_record = []
test_loss_record = []
test_r2_record = []
test_loss_l1_record = []

# for epoch in tqdm(range(1, config.epoch + 1)):
for epoch in (range(300)):
  repres_list = []
  label_list = []

  for idx, (disease, mirna, label, pre_hair_x, _, _) in (enumerate(dataloader)):
    disease = disease.cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    

    digits = model_class(disease, pre_hair_x)
    # digits = torch.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.float32).cuda()
    L1_loss = criterion_L1(digits, label)
    loss = criterion(digits, label)
    

    # print('torch.round(output): ', torch.round(output))
    
    
    optimizer.zero_grad()
    # if epoch < 10:
    #   loss.backward()
    # else:
    #   L1_loss.backward()
    loss.backward()
    optimizer.step()
    steps += 1
  
  train_loss_record.append(loss.item())
  train_loss_l1_record.append(L1_loss.item())

  val_loss, val_acc, val_r2, val_l1 = val_acc_output(epoch)
  # step_log_interval.append(steps)
  # train_acc_record.append(train_acc)
  val_loss_record.append(val_loss)
  val_r2_record.append(val_r2)
  val_loss_l1_record.append(val_l1)
  # train_loss_record.append(loss)
  test_loss, test_acc, test_r2, test_l1 = test_acc_output(epoch)
  # test_acc_record.append(test_acc)
  test_loss_record.append(test_loss)
  test_r2_record.append(test_r2)
  test_loss_l1_record.append(test_l1)
  # break
    

Testing Epoch[0] Loss:0.3990931920707226 | L1 Loss:0.4644263070076704 | R2:-0.08394585627444964 | ACC: 59.8000%(299/500)
Testing Epoch[0] Loss:0.40960417985916137 | L1 Loss:0.4619806706905365 | R2:-0.1381687335255278 | ACC: 60.6667%(182/300)
Testing Epoch[1] Loss:0.3699309639632702 | L1 Loss:0.4571790359914303 | R2:-0.010883277402754243 | ACC: 59.0000%(295/500)
Testing Epoch[1] Loss:0.38487506210803984 | L1 Loss:0.46475850939750674 | R2:-0.08796261573408222 | ACC: 61.0000%(183/300)
Testing Epoch[2] Loss:0.355099992826581 | L1 Loss:0.44960989244282246 | R2:0.0300983987307607 | ACC: 60.4000%(302/500)
Testing Epoch[2] Loss:0.3900176078081131 | L1 Loss:0.4690806299448013 | R2:-0.11088780898064506 | ACC: 60.3333%(181/300)
Testing Epoch[3] Loss:0.34383909683674574 | L1 Loss:0.44617580249905586 | R2:0.05798540746631667 | ACC: 60.0000%(300/500)
Testing Epoch[3] Loss:0.3817059278488159 | L1 Loss:0.4666066259145737 | R2:-0.09137921886507354 | ACC: 60.3333%(181/300)
Testing Epoch[4] Loss:0.345993

KeyboardInterrupt: ignored

In [ ]:
print('-----MLP+ bert 150 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record))

-----MLP+ bert 150 epochs-----
max test_r2_record  -0.05101144358639309
min test_loss_l1_record  0.4499931424856186
min test_loss_record  0.36704154014587403


## LSTM

In [ ]:
model_class = Classificaion_module().cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-300, 300])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=32)
val_dataloader = DataLoader(val_dataset, batch_size=32)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
optimizer = torch.optim.AdamW(params=model_class.parameters(), lr=0.0008, weight_decay=0.0001)
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
criterion_L1 = nn.L1Loss()

In [ ]:
model_class.train()
model_class

Classificaion_module(
  (lstm_module): LSTM_module(
    (lstm1): LSTM(300, 64, num_layers=2, dropout=0.5, bidirectional=True)
    (FC): Linear(in_features=128, out_features=64, bias=True)
  )
  (linear_1): Linear(in_features=70, out_features=128, bias=True)
  (bn_1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_2): Linear(in_features=128, out_features=256, bias=True)
  (bn_2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_3): Linear(in_features=256, out_features=128, bias=True)
  (bn_3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_4): Linear(in_features=128, out_features=64, bias=True)
  (bn_4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (FC1): Linear(in_features=128, out_features=64, bias=True)
  (RELU): ReLU()
  (fc_bn_1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affi

In [ ]:
def test_acc_output(epoch):
  corrects = 0
  the_batch_size = 0

  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0

  # state = default_evaluator.run([[y_pred, y_true]])
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(test_dataloader):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    digits = model_class(disease, pre_hair_x)
    # digits = F.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)
    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTesting Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1
  # corrects = (torch.max(logits, 1)[1] == label).sum()

In [ ]:
def val_acc_output(epoch):
  corrects = 0
  the_batch_size = 0
  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(val_dataloader):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    digits = model_class(disease, pre_hair_x)

    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)

    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTesting Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1

In [ ]:
steps = 0
best_acc = 0
best_performance = 0
step_log_interval = []
train_loss_record = []
train_loss_l1_record = []

val_acc_record = []
val_loss_record = []
val_r2_record = []
val_loss_l1_record = []

test_acc_record = []
test_loss_record = []
test_r2_record = []
test_loss_l1_record = []

# for epoch in tqdm(range(1, config.epoch + 1)):
for epoch in (range(300)):
  repres_list = []
  label_list = []

  for idx, (disease, mirna, label, pre_hair_x, _, _) in (enumerate(dataloader)):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    

    digits = model_class(disease, pre_hair_x)
    # digits = torch.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.float32).cuda()
    L1_loss = criterion_L1(digits, label)
    loss = criterion(digits, label)
    

    # print('torch.round(output): ', torch.round(output))
    
    
    optimizer.zero_grad()
    # if epoch < 10:
    #   loss.backward()
    # else:
    #   L1_loss.backward()
    loss.backward()
    optimizer.step()
    steps += 1
  
  train_loss_record.append(loss.item())
  train_loss_l1_record.append(L1_loss.item())

  val_loss, val_acc, val_r2, val_l1 = val_acc_output(epoch)
  # step_log_interval.append(steps)
  # train_acc_record.append(train_acc)
  val_loss_record.append(val_loss)
  val_r2_record.append(val_r2)
  val_loss_l1_record.append(val_l1)
  # train_loss_record.append(loss)
  test_loss, test_acc, test_r2, test_l1 = test_acc_output(epoch)
  # test_acc_record.append(test_acc)
  test_loss_record.append(test_loss)
  test_r2_record.append(test_r2)
  test_loss_l1_record.append(test_l1)
  # break
    

Testing Epoch[0] Loss:0.3720568921416998 | L1 Loss:0.4487047288566828 | R2:-0.14343281799212865 | ACC: 62.8000%(314/500)
Testing Epoch[0] Loss:0.4493206560611725 | L1 Loss:0.5045110434293747 | R2:-0.27687010614751745 | ACC: 57.6667%(173/300)
Testing Epoch[1] Loss:0.3544533089734614 | L1 Loss:0.4435701249167323 | R2:-0.07573517933320242 | ACC: 63.8000%(319/500)
Testing Epoch[1] Loss:0.4319689333438873 | L1 Loss:0.49116329848766327 | R2:-0.24031011562686738 | ACC: 59.3333%(178/300)
Testing Epoch[2] Loss:0.3364772880449891 | L1 Loss:0.4296735171228647 | R2:-0.02416132140472422 | ACC: 64.4000%(322/500)
Testing Epoch[2] Loss:0.4093720048666 | L1 Loss:0.4841644257307053 | R2:-0.16989128098046974 | ACC: 59.0000%(177/300)
Testing Epoch[3] Loss:0.34587516728788614 | L1 Loss:0.43987264949828386 | R2:-0.053868918878471145 | ACC: 65.2000%(326/500)
Testing Epoch[3] Loss:0.40683853328228 | L1 Loss:0.48171410858631136 | R2:-0.16360830483654484 | ACC: 60.3333%(181/300)
Testing Epoch[4] Loss:0.31825291

In [ ]:
steps = 0
best_acc = 0
best_performance = 0
step_log_interval = []
# train_loss_record = []
# train_loss_l1_record = []

# val_acc_record = []
# val_loss_record = []
# val_r2_record = []
# val_loss_l1_record = []

# test_acc_record = []
# test_loss_record = []
# test_r2_record = []
# test_loss_l1_record = []

# for epoch in tqdm(range(1, config.epoch + 1)):
for epoch in (range(200)):
  repres_list = []
  label_list = []

  for idx, (disease, mirna, label, pre_hair_x, _, _) in (enumerate(dataloader)):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    

    digits = model_class(disease, pre_hair_x)
    # digits = torch.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.float32).cuda()
    L1_loss = criterion_L1(digits, label)
    loss = criterion(digits, label)
    

    # print('torch.round(output): ', torch.round(output))
    
    
    optimizer.zero_grad()
    # if epoch < 10:
    #   loss.backward()
    # else:
    #   L1_loss.backward()
    loss.backward()
    optimizer.step()
    steps += 1
  
  train_loss_record.append(loss.item())
  train_loss_l1_record.append(L1_loss.item())

  val_loss, val_acc, val_r2, val_l1 = val_acc_output(epoch)
  # step_log_interval.append(steps)
  # train_acc_record.append(train_acc)
  val_loss_record.append(val_loss)
  val_r2_record.append(val_r2)
  val_loss_l1_record.append(val_l1)
  # train_loss_record.append(loss)
  test_loss, test_acc, test_r2, test_l1 = test_acc_output(epoch)
  # test_acc_record.append(test_acc)
  test_loss_record.append(test_loss)
  test_r2_record.append(test_r2)
  test_loss_l1_record.append(test_l1)
  # break
    

Testing Epoch[0] Loss:0.15229684673249722 | L1 Loss:0.27958262991160154 | R2:0.5086700231610033 | ACC: 84.2000%(421/500)
Testing Epoch[0] Loss:0.34902486503124236 | L1 Loss:0.4402288913726807 | R2:0.007116759045592014 | ACC: 65.6667%(197/300)
Testing Epoch[1] Loss:0.1500787097029388 | L1 Loss:0.27834083419293165 | R2:0.5080123629887561 | ACC: 84.4000%(422/500)
Testing Epoch[1] Loss:0.3603450655937195 | L1 Loss:0.44200466871261596 | R2:-0.021905257616711648 | ACC: 65.6667%(197/300)
Testing Epoch[2] Loss:0.1460945352446288 | L1 Loss:0.2776883551850915 | R2:0.5329150974485188 | ACC: 84.6000%(423/500)
Testing Epoch[2] Loss:0.36564385890960693 | L1 Loss:0.4510753840208054 | R2:-0.05740901310624112 | ACC: 64.3333%(193/300)
Testing Epoch[3] Loss:0.14615087979473174 | L1 Loss:0.2750239819288254 | R2:0.5351700322940559 | ACC: 85.0000%(425/500)
Testing Epoch[3] Loss:0.35748870074748995 | L1 Loss:0.44332456290721894 | R2:-0.038275695233009145 | ACC: 66.0000%(198/300)
Testing Epoch[4] Loss:0.13999

In [ ]:
print('-----MLP+ LSTM 300 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record))
# -----MLP+ LSTM 300 epochs-----
# max val_r2_record  0.5580382648926783
# max test_r2_record  0.07176489750843243
# min val_loss_l1_record  0.26480430364608765
# min test_loss_l1_record  0.41867785453796386
# min val_loss_record  0.13632171461358666
# min test_loss_record  0.33112146854400637

-----MLP+ LSTM 300 epochs-----
max test_r2_record  0.07176489750843243
min test_loss_l1_record  0.41867785453796386
min test_loss_record  0.33112146854400637


In [ ]:
print('-----MLP+ LSTM 500 epochs-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record))

# print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record))

# print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record))
# -----MLP+ LSTM 500 epochs-----
# max val_r2_record  0.5580382648926783
# max test_r2_record  0.07176489750843243
# min val_loss_l1_record  0.26480430364608765
# min test_loss_l1_record  0.41867785453796386
# min val_loss_record  0.13632171461358666
# min test_loss_record  0.33112146854400637

-----MLP+ LSTM 500 epochs-----
max test_r2_record  0.07176489750843243
min test_loss_l1_record  0.41867785453796386
min test_loss_record  0.33112146854400637


## GRU

In [ ]:
model_class = Classificaion_module().cuda()
dataset = RegularDataset(x_disease, x_rna, y_soft, hair_x)
train_set, test_set = torch.utils.data.random_split(dataset, [len(dataset)-300, 300])
val_dataset, _ = torch.utils.data.random_split(train_set, [500,len(train_set)-500])
dataloader = DataLoader(train_set, batch_size=32)
test_dataloader = DataLoader(test_set, batch_size=32)
val_dataloader = DataLoader(val_dataset, batch_size=32)

In [ ]:
torch.cuda.is_available()

True

In [ ]:
optimizer = torch.optim.AdamW(params=model_class.parameters(), lr=0.0008, weight_decay=0.0001)
# criterion = nn.CrossEntropyLoss()
criterion = nn.MSELoss()
criterion_L1 = nn.L1Loss()

In [ ]:
model_class.train()
model_class

Classificaion_module(
  (gru_module): GRU_module(
    (gru): GRU(300, 64, num_layers=2, dropout=0.5, bidirectional=True)
    (FC): Linear(in_features=128, out_features=64, bias=True)
  )
  (linear_1): Linear(in_features=70, out_features=128, bias=True)
  (bn_1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_2): Linear(in_features=128, out_features=256, bias=True)
  (bn_2): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_3): Linear(in_features=256, out_features=128, bias=True)
  (bn_3): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear_4): Linear(in_features=128, out_features=64, bias=True)
  (bn_4): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (FC1): Linear(in_features=128, out_features=64, bias=True)
  (RELU): ReLU()
  (fc_bn_1): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=Tr

In [ ]:
def test_acc_output(epoch):
  corrects = 0
  the_batch_size = 0

  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0

  # state = default_evaluator.run([[y_pred, y_true]])
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(test_dataloader):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    digits = model_class(disease, pre_hair_x)
    # digits = F.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)
    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rTesting Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1
  # corrects = (torch.max(logits, 1)[1] == label).sum()

In [ ]:
def val_acc_output(epoch):
  corrects = 0
  the_batch_size = 0
  losses = 0
  losses_l1 = 0
  r2s = 0
  c = 0
  for idx, (disease, mirna, label, pre_hair_x, _, _) in enumerate(val_dataloader):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    digits = model_class(disease, pre_hair_x)

    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.LongTensor).cuda()
    loss = criterion(digits, label)
    L1_loss = criterion_L1(digits, label)

    r2 = r2_score(label.cpu().detach().numpy(), digits.cpu().detach().numpy())
    corrects += (torch.round(digits) == label).sum()
    the_batch_size += label.shape[0]
    c += 1
    losses += loss.item()
    losses_l1 += L1_loss.item()
    r2s += r2
  acc = 100.0 * corrects / the_batch_size
  avg_loss = losses/c
  avg_loss_l1 = losses_l1/c
  avg_r2 = r2s/c
  print('\rValidation Epoch[{}] Loss:{} | L1 Loss:{} | R2:{} | ACC: {:.4f}%({}/{})'.format(epoch, avg_loss, avg_loss_l1, avg_r2, acc, corrects, the_batch_size))

  return avg_loss, acc, avg_r2, avg_loss_l1

In [ ]:
steps = 0
best_acc = 0
best_performance = 0
step_log_interval = []
train_loss_record = []
train_loss_l1_record = []

val_acc_record = []
val_loss_record = []
val_r2_record = []
val_loss_l1_record = []

test_acc_record = []
test_loss_record = []
test_r2_record = []
test_loss_l1_record = []

# for epoch in tqdm(range(1, config.epoch + 1)):
for epoch in (range(300)):
  repres_list = []
  label_list = []

  for idx, (disease, mirna, label, pre_hair_x, _, _) in (enumerate(dataloader)):
    disease = disease.to(torch.float32).cuda()
    pre_hair_x = pre_hair_x.cuda()
    label = label.cuda()
    

    digits = model_class(disease, pre_hair_x)
    # digits = torch.tanh(digits)
    # print('output: ', output.shape)
    digits = digits.to(torch.float32).squeeze()
    label = label.type(torch.float32).cuda()
    L1_loss = criterion_L1(digits, label)
    loss = criterion(digits, label)
    

    # print('torch.round(output): ', torch.round(output))
    
    
    optimizer.zero_grad()
    # if epoch < 10:
    #   loss.backward()
    # else:
    #   L1_loss.backward()
    loss.backward()
    optimizer.step()
    steps += 1
  
  train_loss_record.append(loss.item())
  train_loss_l1_record.append(L1_loss.item())

  val_loss, val_acc, val_r2, val_l1 = val_acc_output(epoch)
  # step_log_interval.append(steps)
  # train_acc_record.append(train_acc)
  val_loss_record.append(val_loss)
  val_r2_record.append(val_r2)
  val_loss_l1_record.append(val_l1)
  # train_loss_record.append(loss)
  test_loss, test_acc, test_r2, test_l1 = test_acc_output(epoch)
  # test_acc_record.append(test_acc)
  test_loss_record.append(test_loss)
  test_r2_record.append(test_r2)
  test_loss_l1_record.append(test_l1)
  # break
    

Validation Epoch[0] Loss:0.41209696512669325 | L1 Loss:0.47777600958943367 | R2:-0.11327524909348965 | ACC: 59.4000%(297/500)
Testing Epoch[0] Loss:0.39360188096761706 | L1 Loss:0.46407740116119384 | R2:-0.07748322004312877 | ACC: 61.0000%(183/300)
Validation Epoch[1] Loss:0.3861011303961277 | L1 Loss:0.46569998376071453 | R2:-0.04470996121859471 | ACC: 60.6000%(303/500)
Testing Epoch[1] Loss:0.3771457850933075 | L1 Loss:0.45682312548160553 | R2:-0.041832035404083055 | ACC: 61.0000%(183/300)
Validation Epoch[2] Loss:0.3696885034441948 | L1 Loss:0.4593101851642132 | R2:0.00034636283487322894 | ACC: 61.0000%(305/500)
Testing Epoch[2] Loss:0.39648000001907346 | L1 Loss:0.4719322234392166 | R2:-0.1009981632345482 | ACC: 59.6667%(179/300)
Validation Epoch[3] Loss:0.35904973465949297 | L1 Loss:0.45943489857017994 | R2:0.028669554120834315 | ACC: 62.8000%(314/500)
Testing Epoch[3] Loss:0.3897454857826233 | L1 Loss:0.47708634436130526 | R2:-0.07595643492846212 | ACC: 60.3333%(181/300)
Validati

In [ ]:
print('-----MLP+ GRU 300 epochs ver1-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record))

# # print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record))

# # print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record))
# -----MLP+ GRU 300 epochs-----
# max val_r2_record  0.5408867360567617
# max test_r2_record  0.27864913230701527
# min val_loss_l1_record  0.2862258832901716
# min test_loss_l1_record  0.3514682978391647
# min val_loss_record  0.16164700035005808
# min test_loss_record  0.25647406876087187

-----MLP+ GRU 300 epochs-----
max test_r2_record  0.27864913230701527
min test_loss_l1_record  0.3514682978391647
min test_loss_record  0.25647406876087187


In [ ]:
print('-----MLP+ GRU 200 epochs ver2-----')
# print('max val_r2_record ', max(val_r2_record))
print('max test_r2_record ', max(test_r2_record[:200]))

# # print('min val_loss_l1_record ', min(val_loss_l1_record))
print('min test_loss_l1_record ', min(test_loss_l1_record[:200]))

# # print('min val_loss_record ', min(val_loss_record))
print('min test_loss_record ', min(test_loss_record[:200]))
# -----MLP+ GRU 300 epochs-----
# max val_r2_record  0.5408867360567617
# max test_r2_record  0.27864913230701527
# min val_loss_l1_record  0.2862258832901716
# min test_loss_l1_record  0.3514682978391647
# min val_loss_record  0.16164700035005808
# min test_loss_record  0.25647406876087187

-----MLP+ GRU 200 epochs ver2-----
max test_r2_record  0.324528637632052
min test_loss_l1_record  0.3708655506372452
min test_loss_record  0.23711877018213273
